In [1]:
import numpy as np
import bcolz
import h5py
import os
os.chdir('../..')
import allel
allel.__version__

'0.20.3'

In [2]:
a = np.arange(20000000)
a

array([       0,        1,        2, ..., 19999997, 19999998, 19999999])

In [3]:
c = allel.chunked.bcolztmp_storage.array(a, cparams=bcolz.cparams(cname='zlib', clevel=1, shuffle=True))
c.flush()
c

carray((20000000,), int64)
  nbytes := 152.59 MB; cbytes := 2.41 MB; ratio: 63.28
  cparams := cparams(clevel=1, shuffle=True, cname='zlib', quantize=0)
  chunklen := 131072; chunksize: 1048576; blocksize: 131072
  rootdir := '/tmp/scikit_allel_znyqdfcp.bcolz'
  mode    := 'w'
[       0        1        2 ..., 19999997 19999998 19999999]

In [4]:
!du -hs {c.rootdir}

1.9M	/tmp/scikit_allel_znyqdfcp.bcolz


In [5]:
allel.chunked.ChunkedArray(c)

ChunkedArray((20000000,), int64, chunks=(131072,))
  nbytes: 152.6M; cbytes: 2.4M; cratio: 63.3;
  compression: blosc; compression_opts: cparams(clevel=1, shuffle=True, cname='zlib', quantize=0);
  data: bcolz.carray_ext.carray

In [6]:
h = allel.chunked.hdf5tmp_storage.array(a, compression='gzip', compression_opts=1, shuffle=True, chunks=(c.chunklen,))
h.file.flush()
h

<HDF5 dataset "data": shape (20000000,), type "<i8">

In [7]:
!du -h {h.file.filename}

932K	/tmp/scikit_allel_s60rnepu.h5


In [8]:
allel.chunked.ChunkedArray(h)

ChunkedArray((20000000,), int64, chunks=(131072,))
  nbytes: 152.6M; cbytes: 920.4K; cratio: 169.8;
  compression: gzip; compression_opts: 1;
  data: h5py._hl.dataset.Dataset

In [9]:
z = allel.chunked.copy(a, storage='zarrmem')
z

zarr.core.Array((20000000,), int64, chunks=(131072,), order=C)
  compression: blosc; compression_opts: {'clevel': 5, 'shuffle': 1, 'cname': 'blosclz'}
  nbytes: 152.6M; nbytes_stored: 2.7M; ratio: 57.0; initialized: 153/153
  store: builtins.dict

In [10]:
z = allel.chunked.copy(a, storage='zarrtmp')
z

zarr.core.Array((20000000,), int64, chunks=(131072,), order=C)
  compression: blosc; compression_opts: {'clevel': 5, 'shuffle': 1, 'cname': 'blosclz'}
  nbytes: 152.6M; nbytes_stored: 2.7M; ratio: 57.0; initialized: 153/153
  store: zarr.storage.DirectoryStore

In [11]:
z.store.path

'/tmp/scikit_allel_jclr1o41.zarr'

In [12]:
z = allel.chunked.copy(a, storage='zarr', path='array.zarr')
z

zarr.core.Array((20000000,), int64, chunks=(131072,), order=C)
  compression: blosc; compression_opts: {'clevel': 5, 'shuffle': 1, 'cname': 'blosclz'}
  nbytes: 152.6M; nbytes_stored: 2.7M; ratio: 57.0; initialized: 153/153
  store: zarr.storage.DirectoryStore

In [13]:
z.store.path

'/home/aliman/src/github/cggh/scikit-allel/array.zarr'

In [14]:
g = allel.chunked.hdf5tmp_storage.table([a, a], compression='gzip', compression_opts=1, shuffle=True)
g

<HDF5 file "scikit_allel_hyd2kn7g.h5" (mode r+)>

In [15]:
allel.chunked.ChunkedTable(g)

f0,f1
0,0
1,1
2,2
3,3
4,4


In [18]:
allel.chunked.copy_table(g, storage='bcolz')

ctable((20000000,), [('f0', '<i8'), ('f1', '<i8')])
  nbytes: 305.18 MB; cbytes: 7.19 MB; ratio: 42.44
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
[(0, 0) (1, 1) (2, 2) ..., (19999997, 19999997) (19999998, 19999998)
 (19999999, 19999999)]

In [19]:
z = allel.chunked.copy_table(g, storage='zarr')
z.names, z.columns

(['f0', 'f1'], [zarr.core.Array((20000000,), int64, chunks=(131072,), order=C)
    compression: blosc; compression_opts: {'clevel': 5, 'shuffle': 1, 'cname': 'blosclz'}
    nbytes: 152.6M; nbytes_stored: 2.7M; ratio: 57.0; initialized: 153/153
    store: builtins.dict,
  zarr.core.Array((20000000,), int64, chunks=(131072,), order=C)
    compression: blosc; compression_opts: {'clevel': 5, 'shuffle': 1, 'cname': 'blosclz'}
    nbytes: 152.6M; nbytes_stored: 2.7M; ratio: 57.0; initialized: 153/153
    store: builtins.dict])

In [20]:
z = allel.chunked.copy_table(g, storage='zarr', path='test.zarr')
z.names, z.columns

(['f0', 'f1'], [zarr.core.Array((20000000,), int64, chunks=(131072,), order=C)
    compression: blosc; compression_opts: {'clevel': 5, 'shuffle': 1, 'cname': 'blosclz'}
    nbytes: 152.6M; nbytes_stored: 2.7M; ratio: 57.0; initialized: 153/153
    store: zarr.storage.DirectoryStore,
  zarr.core.Array((20000000,), int64, chunks=(131072,), order=C)
    compression: blosc; compression_opts: {'clevel': 5, 'shuffle': 1, 'cname': 'blosclz'}
    nbytes: 152.6M; nbytes_stored: 2.7M; ratio: 57.0; initialized: 153/153
    store: zarr.storage.DirectoryStore])

In [21]:
!ls -lh test.zarr/

total 8.0K
drwxrwxr-x 2 aliman aliman 4.0K Jul 13 23:18 f0
drwxrwxr-x 2 aliman aliman 4.0K Jul 13 23:18 f1


In [22]:
!ls -lh test.zarr/f0 | head -n10

total 3.0M
-rw------- 1 aliman aliman 18K Jul 13 23:18 0
-rw------- 1 aliman aliman 18K Jul 13 23:18 1
-rw------- 1 aliman aliman 18K Jul 13 23:18 10
-rw------- 1 aliman aliman 18K Jul 13 23:18 100
-rw------- 1 aliman aliman 18K Jul 13 23:18 101
-rw------- 1 aliman aliman 18K Jul 13 23:18 102
-rw------- 1 aliman aliman 18K Jul 13 23:18 103
-rw------- 1 aliman aliman 18K Jul 13 23:18 104
-rw------- 1 aliman aliman 18K Jul 13 23:18 105


In [23]:
z = allel.chunked.copy_table(g, storage='zarrtmp')
z.names, z.columns

(['f0', 'f1'], [zarr.core.Array((20000000,), int64, chunks=(131072,), order=C)
    compression: blosc; compression_opts: {'clevel': 5, 'shuffle': 1, 'cname': 'blosclz'}
    nbytes: 152.6M; nbytes_stored: 2.7M; ratio: 57.0; initialized: 153/153
    store: zarr.storage.DirectoryStore,
  zarr.core.Array((20000000,), int64, chunks=(131072,), order=C)
    compression: blosc; compression_opts: {'clevel': 5, 'shuffle': 1, 'cname': 'blosclz'}
    nbytes: 152.6M; nbytes_stored: 2.7M; ratio: 57.0; initialized: 153/153
    store: zarr.storage.DirectoryStore])

In [24]:
z['f0'].store.path

'/tmp/scikit_allel_42hpnru_.zarr/f0'

In [25]:
allel.chunked.ChunkedTable(z)

f0,f1
0,0
1,1
2,2
3,3
4,4
